In [262]:
import os
import sys 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
lebron_db_path = r"../LEBRON/lebron.db"
jordan_db_path = r"../JORDAN/jordan.db"

In [124]:
filepath = '../JORDAN/team_adv_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN//team_adv_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_advanced_team_stats = pd.concat(data).drop(columns=['Rk'])

jordan_advanced_team_stats.columns = [col.lower() for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('2', 'two_') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('3', 'three_') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('%', '_perc') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('/48', 'per48') for col in jordan_advanced_team_stats]
jordan_advanced_team_stats.columns = [col.replace('-', '_') for col in jordan_advanced_team_stats]

names = jordan_advanced_team_stats['player'].values
ages = jordan_advanced_team_stats['age'].values

filepath = '../JORDAN/team_per_pos_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN//team_per_pos_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_per_100_possessions_team_stats = pd.concat(data).drop(columns=['Rk'])

jordan_per_100_possessions_team_stats.columns = [col.lower() for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('2', 'two_') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('3', 'three_') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('%', '_perc') for col in jordan_per_100_possessions_team_stats]
jordan_per_100_possessions_team_stats.columns = [col.replace('-', '_') for col in jordan_per_100_possessions_team_stats]

names = jordan_per_100_possessions_team_stats['player'].values
ages = jordan_per_100_possessions_team_stats['age'].values

filepath = '../JORDAN/game_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../JORDAN/game_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

jordan_game_data = pd.concat(data).drop(columns=['Unnamed: 5', 'Rk', 'Gcar', 'Gtm'])
jordan_game_data['GS'] = jordan_game_data['GS'].apply(lambda x: 1 if x == '*' else 0)

jordan_game_data.columns = [col.lower() for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('2', 'two_') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('3', 'three_') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('%', '_perc') for col in jordan_game_data.columns]
jordan_game_data.columns = [col.replace('+/-', 'bpm') for col in jordan_game_data.columns]

result = jordan_game_data['result'].str.slice(0, 1)
result = [0 if r == 'L' else 1 for r in result]
scores = jordan_game_data['result'].str.split('-')

home_scores = []
away_scores = []

for score in scores:
    home_score = score[0][2:]
    away_score = score[1][0:3].strip()

    home_scores.append(home_score)
    away_scores.append(away_score)

jordan_game_data['result'] = result
jordan_game_data['team_score'] = home_scores
jordan_game_data['opp_score'] = away_scores

times = jordan_game_data['mp'].str.split(':')

seconds = []
for t in times:
    min_sec = int(t[0]) * 60
    time = min_sec + int(t[1])
    
    seconds.append(time)

jordan_game_data['mp'] = seconds

In [125]:
filepath = '../LEBRON/game_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//game_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_game_data = pd.concat(data).drop(columns=['Unnamed: 5', 'Rk', 'Gcar', 'Gtm'])
lebron_game_data['GS'] = lebron_game_data['GS'].apply(lambda x: 1 if x == '*' else 0)

lebron_game_data.columns = [col.lower() for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('2', 'two_') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('3', 'three_') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('%', '_perc') for col in lebron_game_data.columns]
lebron_game_data.columns = [col.replace('+/-', 'bpm') for col in lebron_game_data.columns]

result = lebron_game_data['result'].str.slice(0, 1)
result = [0 if r == 'L' else 1 for r in result]
scores = lebron_game_data['result'].str.split('-')

home_scores = []
away_scores = []

for score in scores:
    home_score = score[0][2:]
    away_score = score[1][0:3].strip()

    home_scores.append(home_score)
    away_scores.append(away_score)

lebron_game_data['result'] = result
lebron_game_data['team_score'] = home_scores
lebron_game_data['opp_score'] = away_scores

times = lebron_game_data['mp'].str.split(':')

seconds = []
for t in times:
    min_sec = int(t[0]) * 60
    time = min_sec + int(t[1])
    
    seconds.append(time)

lebron_game_data['mp'] = seconds

filepath = '../LEBRON/team_per_pos_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//team_per_pos_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_per_100_possessions_team_stats = pd.concat(data).drop(columns=['Rk'])

lebron_per_100_possessions_team_stats.columns = [col.lower() for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('2', 'two_') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('3', 'three_') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('%', '_perc') for col in lebron_per_100_possessions_team_stats]
lebron_per_100_possessions_team_stats.columns = [col.replace('-', '_') for col in lebron_per_100_possessions_team_stats]

names = lebron_per_100_possessions_team_stats['player'].values
ages = lebron_per_100_possessions_team_stats['age'].values

filepath = '../LEBRON/team_adv_data'
contents = os.listdir(filepath)

data = []
for file in contents:
    file_path = f'../LEBRON//team_adv_data/{file}'
    df = pd.read_csv(file_path)
    df['season'] = file.split('.')[0]
    data.append(df)

lebron_advanced_team_stats = pd.concat(data).drop(columns=['Rk'])

lebron_advanced_team_stats.columns = [col.lower() for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('2', 'two_') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('3', 'three_') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('%', '_perc') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('/48', 'per48') for col in lebron_advanced_team_stats]
lebron_advanced_team_stats.columns = [col.replace('-', '_') for col in lebron_advanced_team_stats]

names = lebron_advanced_team_stats['player'].values
ages = lebron_advanced_team_stats['age'].values


In [126]:
###
# For some reason not all data is present when using database 12.26
###

# with SQLiteClient(jordan_db_path) as db:
#     jordan_game_data = pd.DataFrame(
#         db.query("SELECT * FROM jordan_game_data")
#     )
#     jordan_per_100_possessions_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM jordan_per_100_possessions_team_stats")
#     )
#     jordan_advanced_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM jordan_advanced_team_stats")
#     )

# with SQLiteClient(lebron_db_path) as db:
#     lebron_game_data = pd.DataFrame(
#         db.query("SELECT * FROM lebron_game_data")
#     )
#     lebron_per_100_possessions_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM lebron_per_100_possessions_team_stats")
#     )
#     lebron_advanced_team_stats = pd.DataFrame(
#         db.query("SELECT * FROM lebron_advanced_team_stats")
#     )

lebron_regular_game_data = lebron_game_data[
    ~lebron_game_data['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_game_data = lebron_game_data[
    lebron_game_data['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_game_data = jordan_game_data[
    ~jordan_game_data['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_game_data = jordan_game_data[
    jordan_game_data['season'].str.contains('playoff')
].reset_index(drop=True)

lebron_regular_100_possession_team = lebron_per_100_possessions_team_stats[
    ~lebron_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_100_possession_team = lebron_per_100_possessions_team_stats[
    lebron_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_100_possession_team = jordan_per_100_possessions_team_stats[
    ~jordan_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_100_possession_team = jordan_per_100_possessions_team_stats[
    jordan_per_100_possessions_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

lebron_regular_advanced_team_stats = lebron_advanced_team_stats[
    ~lebron_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
lebron_playoff_advanced_team_stats = lebron_advanced_team_stats[
    lebron_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

jordan_regular_advanced_team_stats = jordan_advanced_team_stats[
    ~jordan_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)
jordan_playoff_advanced_team_stats = jordan_advanced_team_stats[
    jordan_advanced_team_stats['season'].str.contains('playoff')
].reset_index(drop=True)

In [127]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = jordan_playoff_game_data['date'].values
colz = pd.unique(
    jordan_playoff_advanced_team_stats[
        jordan_playoff_advanced_team_stats['player'] != 'Michael Jordan'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = jordan_playoff_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = jordan_playoff_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = jordan_playoff_advanced_team_stats[
        (jordan_playoff_advanced_team_stats['player'] != 'Michael Jordan') &
        (jordan_playoff_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

jordan_playoff_game_data = pd.concat([jordan_playoff_game_data, player_matrix], axis=1)

In [128]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = jordan_regular_game_data['date'].values
colz = pd.unique(
    jordan_regular_advanced_team_stats[
        jordan_regular_advanced_team_stats['player'] != 'Michael Jordan'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = jordan_regular_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = jordan_regular_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = jordan_regular_advanced_team_stats[
        (jordan_regular_advanced_team_stats['player'] != 'Michael Jordan') &
        (jordan_regular_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

jordan_regular_game_data = pd.concat([jordan_regular_game_data, player_matrix], axis=1)

In [129]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = lebron_playoff_game_data['date'].values
colz = pd.unique(
    lebron_playoff_advanced_team_stats[
        lebron_playoff_advanced_team_stats['player'] != 'LeBron James'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = lebron_playoff_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = lebron_playoff_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = lebron_playoff_advanced_team_stats[
        (lebron_playoff_advanced_team_stats['player'] != 'LeBron James') &
        (lebron_playoff_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

lebron_playoff_game_data = pd.concat([lebron_playoff_game_data, player_matrix], axis=1)

In [130]:
###
# Creating a player matrix of every teammate 
# Jordan had during every playoff series game
###

rows = lebron_regular_game_data['date'].values
colz = pd.unique(
    lebron_regular_advanced_team_stats[
        lebron_regular_advanced_team_stats['player'] != 'LeBron James'
    ]['player']
)

cols = {}
for col in colz:
    cols[col] = np.zeros(len(rows))

player_matrix = pd.DataFrame(cols, index=rows).reset_index().rename(columns={'index':'date'})
player_matrix['season'] = lebron_regular_game_data['season'].values

###
# Updating the player value to 1 if they played
# with Jordan in a game (or on his team)
###

test = lebron_regular_advanced_team_stats[['player', 'season']]
seasons = pd.unique(test['season'])

for season in seasons:
    sample = lebron_regular_advanced_team_stats[
        (lebron_regular_advanced_team_stats['player'] != 'LeBron James') &
        (lebron_regular_advanced_team_stats['season'] == season)
    ]

    players = sample['player'].values
        
    for i in range(len(player_matrix)):
        if player_matrix.iloc[i]['season'] == season:
            for player in players:
                player_matrix.loc[i, player] = 1

player_matrix = player_matrix.drop(columns=['season', 'date'])

lebron_regular_game_data = pd.concat([lebron_regular_game_data, player_matrix], axis=1)

In [131]:
# Creating dummy variables for opponents

lebron_reg_opps = pd.get_dummies(lebron_regular_game_data['opp'], dtype=int)
lebron_reg_opps.columns = [col + '_opp' for col in lebron_reg_opps.columns]

lebron_playoff_opps = pd.get_dummies(
    lebron_playoff_game_data['opp'],
    dtype=int
)
lebron_playoff_opps.columns = [
    col + '_opp' for col in lebron_playoff_opps.columns
]


jordan_reg_opps = pd.get_dummies(jordan_regular_game_data['opp'], dtype=int)
jordan_reg_opps.columns = [col + '_opp' for col in jordan_reg_opps.columns]

jordan_playoff_opps = pd.get_dummies(
    jordan_playoff_game_data['opp'], 
    dtype=int
)
jordan_playoff_opps.columns = [
    col + '_opp' for col in jordan_playoff_opps.columns
]

# Creating dummy variables for teammates

lebron_reg_team = pd.get_dummies(lebron_regular_game_data['team'], dtype=int)
lebron_reg_team.columns = [col + '_team' for col in lebron_reg_team.columns]

lebron_playoff_team = pd.get_dummies(
    lebron_playoff_game_data['team'], dtype=int
)
lebron_playoff_team.columns = [
    col + '_team' for col in lebron_playoff_team.columns
]

jordan_reg_team = pd.get_dummies(jordan_regular_game_data['team'], dtype=int)
jordan_reg_team.columns = [col + '_team' for col in jordan_reg_team.columns]

jordan_playoff_team = pd.get_dummies(
    jordan_playoff_game_data['team'], 
    dtype=int
)
jordan_playoff_team.columns = [
    col + '_team' for col in jordan_playoff_team.columns
]

# Combining opp and team data

lebron_reg_combined =  pd.concat([lebron_reg_opps, lebron_reg_team], axis=1)
lebron_playoff_combined =  pd.concat(
    [lebron_playoff_opps, lebron_playoff_team], 
    axis=1
)

jordan_reg_combined =  pd.concat([jordan_reg_opps, jordan_reg_team], axis=1)
jordan_playoff_combined =  pd.concat(
    [jordan_playoff_opps, jordan_playoff_team], 
    axis=1
)

# Combining the opp and team data to data

lebron_regular_game_data = pd.concat(
    [lebron_regular_game_data, lebron_reg_combined], 
    axis=1
)
lebron_playoff_game_data = pd.concat(
    [lebron_playoff_game_data, lebron_playoff_combined], 
    axis=1
)

jordan_regular_game_data = pd.concat(
    [jordan_regular_game_data, jordan_reg_combined], 
    axis=1
)
jordan_playoff_game_data = pd.concat(
    [jordan_playoff_game_data, jordan_playoff_combined], 
    axis=1
)

In [132]:
jordan_regular_game_data = jordan_regular_game_data.drop(
    columns=['team', 'opp']
)
jordan_playoff_game_data = jordan_playoff_game_data.drop(
    columns=['team', 'opp']
)
lebron_regular_game_data = lebron_regular_game_data.drop(
    columns=['team', 'opp']
)
lebron_playoff_game_data = lebron_playoff_game_data.drop(
    columns=['team', 'opp']
)

jordan_regular_game_data['date'] = pd.to_datetime(
    jordan_regular_game_data['date']
)
jordan_playoff_game_data['date'] = pd.to_datetime(
    jordan_playoff_game_data['date']
)
lebron_regular_game_data['date'] = pd.to_datetime(
    lebron_regular_game_data['date']
)
lebron_playoff_game_data['date'] = pd.to_datetime(
    lebron_playoff_game_data['date']
)

jordan_regular_game_data = jordan_regular_game_data.fillna(0)
jordan_playoff_game_data = jordan_playoff_game_data.fillna(0)
lebron_regular_game_data = lebron_regular_game_data.fillna(0)
lebron_playoff_game_data = lebron_playoff_game_data.fillna(0)

In [133]:
lebron_playoff_game_data[['mp', 'ast', 'tov', 'pts', 'blk', 'orb', 'drb', 'result', 'bpm']].describe()

,mp,ast,tov,pts,blk,orb,drb,result,bpm
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,2478.527397,7.174658,3.585616,28.386986,0.972603,1.513699,7.486301,0.630137,4.356164
std,282.667891,2.779461,2.058308,7.961089,1.008181,1.353302,2.808275,0.483596,14.070387
min,1461.000000,1.000000,0.000000,7.000000,0.000000,0.000000,1.000000,0.000000,-32.000000
25%,2334.750000,5.000000,2.000000,23.000000,0.000000,0.000000,5.000000,0.000000,-5.000000
50%,2500.500000,7.000000,3.000000,28.000000,1.000000,1.000000,7.000000,1.000000,5.000000
75%,2683.250000,9.000000,5.000000,33.000000,2.000000,2.000000,9.000000,1.000000,14.000000
max,3180.000000,16.000000,10.000000,51.000000,5.000000,8.000000,16.000000,1.000000,46.000000


In [134]:
lebron_regular_game_data[['mp', 'ast', 'tov', 'pts', 'blk', 'orb', 'drb', 'result', 'bpm']].describe()

,mp,ast,tov,pts,blk,orb,drb,result,bpm
count,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000,1562.000000
mean,2267.953265,7.416133,3.502561,27.006402,0.736236,1.151729,6.358515,0.645967,4.730474
std,311.453411,2.965693,1.826296,7.790989,0.890310,1.163463,2.769056,0.478372,12.820486
min,633.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-39.000000
25%,2094.000000,5.000000,2.000000,22.000000,0.000000,0.000000,4.000000,0.000000,-4.000000
50%,2274.000000,7.000000,3.000000,27.000000,1.000000,1.000000,6.000000,1.000000,6.000000
75%,2472.750000,9.000000,5.000000,32.000000,1.000000,2.000000,8.000000,1.000000,14.000000
max,3281.000000,19.000000,11.000000,61.000000,5.000000,7.000000,19.000000,1.000000,39.000000


In [135]:
jordan_playoff_game_data[['mp', 'ast', 'tov', 'pts', 'blk', 'orb', 'drb', 'result', 'bpm']].describe()

,mp,ast,tov,pts,blk,orb,drb,result,bpm
count,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000
mean,2505.212291,5.709497,3.050279,33.446927,0.882682,1.703911,4.731844,0.664804,1.642458
std,231.546898,2.919225,1.885109,8.825476,1.040031,1.400857,2.323704,0.473383,5.415914
min,1740.000000,1.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,-10.000000
25%,2400.000000,4.000000,2.000000,27.500000,0.000000,1.000000,3.000000,0.000000,0.000000
50%,2525.000000,5.000000,3.000000,32.000000,1.000000,1.000000,4.000000,1.000000,0.000000
75%,2640.000000,7.500000,4.000000,39.000000,1.000000,2.000000,6.000000,1.000000,0.000000
max,3420.000000,14.000000,8.000000,63.000000,5.000000,8.000000,15.000000,1.000000,37.000000


In [136]:
jordan_regular_game_data[['mp', 'ast', 'tov', 'pts', 'blk', 'orb', 'drb', 'result', 'bpm']].describe()

,mp,ast,tov,pts,blk,orb,drb,result,bpm
count,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000,1072.000000
mean,2295.310634,5.254664,2.727612,30.123134,0.833022,1.555970,4.667910,0.658582,1.339552
std,342.000264,2.722709,1.733523,9.751427,1.003780,1.442532,2.568533,0.474407,7.218117
min,736.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,-35.000000
25%,2159.500000,3.000000,1.000000,23.000000,0.000000,0.000000,3.000000,0.000000,0.000000
50%,2340.000000,5.000000,3.000000,30.000000,1.000000,1.000000,4.000000,1.000000,0.000000
75%,2520.000000,7.000000,4.000000,36.000000,1.000000,2.000000,6.000000,1.000000,0.000000
max,3360.000000,17.000000,9.000000,69.000000,6.000000,8.000000,14.000000,1.000000,34.000000


In [137]:
jordan_regular_game_data = jordan_regular_game_data.fillna(0)
jordan_playoff_game_data = jordan_playoff_game_data.fillna(0)
lebron_regular_game_data = lebron_regular_game_data.fillna(0)
lebron_playoff_game_data = lebron_playoff_game_data.fillna(0)

In [283]:
month = jordan_regular_game_data['date'].dt.month.values
pts = jordan_regular_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
jordan_sample = sample[(sample['month'] <= 4) | (sample['month'] >= 10)]

month = lebron_regular_game_data['date'].dt.month.values
pts = lebron_regular_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
lebron_sample = sample[(sample['month'] <= 4) | (sample['month'] >= 10)]

lebron_sample = lebron_sample.groupby('month').mean().reset_index()
jordan_sample = jordan_sample.groupby('month').mean().reset_index()

lebron_sample = lebron_sample.rename(columns={'pts': 'LeBron'})
jordan_sample = jordan_sample.rename(columns={'pts': 'Jordan'})


sample = pd.merge(lebron_sample, jordan_sample, on='month')

fig = make_subplots()

fig.add_trace(
    go.Bar(x=sample['month'], y=sample['Jordan'], name='Jordan Avg. Points'),
)

fig.add_trace(
    go.Scatter(x=sample['month'], y=sample['LeBron'], name='LeBron Avg. Points',
               mode='lines+markers'),
)

fig.update_layout(title='Lebron Monthly Regular Season Point Avg. vs. Jordan Regular Season Point Avg.', height=500, width=800)

y_min = sample[['Jordan','LeBron']].min().min()
y_max = sample[['Jordan','LeBron']].max().max()

fig.update_yaxes(range=[y_min - 1, y_max + 1], secondary_y=False)
fig.update_yaxes(range=[y_min, y_max], secondary_y=True)

fig.update_yaxes(title_text='Points')
fig.update_xaxes(title_text='Month')

fig.show()


In [284]:
month = jordan_playoff_game_data['date'].dt.month.values
pts = jordan_playoff_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
jordan_sample = sample[sample['month'].between(4, 6)]

month = lebron_playoff_game_data['date'].dt.month.values
pts = lebron_playoff_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
lebron_sample = sample[sample['month'].between(4, 6)]


lebron_sample = lebron_sample.groupby('month').mean().reset_index()
jordan_sample = jordan_sample.groupby('month').mean().reset_index()

lebron_sample = lebron_sample.rename(columns={'pts': 'LeBron'})
jordan_sample = jordan_sample.rename(columns={'pts': 'Jordan'})


sample = pd.merge(lebron_sample, jordan_sample, on='month')

fig = make_subplots()

fig.add_trace(
    go.Bar(x=sample['month'], y=sample['Jordan'], name='Jordan Avg. Points'),
)

fig.add_trace(
    go.Scatter(x=sample['month'], y=sample['LeBron'], name='LeBron Avg. Points',
               mode='lines+markers'),
)

fig.update_layout(title='Lebron Monthly Playoff Season Point Avg. vs. Jordan Playoff Season Point Avg.', height=500, width=800)

y_min = sample[['Jordan','LeBron']].min().min()
y_max = sample[['Jordan','LeBron']].max().max()

fig.update_yaxes(range=[y_min - 1, y_max + 1], secondary_y=False)
fig.update_yaxes(range=[y_min, y_max], secondary_y=True)

fig.update_yaxes(title_text='Points')
fig.update_xaxes(title_text='Month')

fig.show()


In [321]:
month = jordan_playoff_game_data['date'].dt.month.values
pts = jordan_playoff_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
jordan_sample = sample[sample['month'].between(4, 7)]

month = lebron_playoff_game_data['date'].dt.month.values
pts = lebron_playoff_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
lebron_sample = sample[sample['month'].between(4, 7)]

lebron_sample = lebron_sample.groupby('month').mean().reset_index()
jordan_sample = jordan_sample.groupby('month').mean().reset_index()

lebron_sample = lebron_sample.rename(columns={'pts': 'LeBron'})
jordan_sample = jordan_sample.rename(columns={'pts': 'Jordan'})


sample1 = pd.merge(lebron_sample, jordan_sample, on='month')

month = jordan_regular_game_data['date'].dt.month.values
pts = jordan_regular_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
jordan_sample = sample[(sample['month'] <= 4) | (sample['month'] >= 10)]

month = lebron_regular_game_data['date'].dt.month.values
pts = lebron_regular_game_data['pts'].values

sample = pd.DataFrame({'month': month, 'pts': pts})
lebron_sample = sample[(sample['month'] <= 4) | (sample['month'] >= 10)]

lebron_sample = lebron_sample.groupby('month').mean().reset_index()
jordan_sample = jordan_sample.groupby('month').mean().reset_index()

lebron_sample = lebron_sample.rename(columns={'pts': 'LeBron'})
jordan_sample = jordan_sample.rename(columns={'pts': 'Jordan'})


sample2 = pd.merge(lebron_sample, jordan_sample, on='month')

sample1['month'] = sample1['month'] + 1

sample = pd.concat([sample1, sample2])

sample = sample.sort_values('month')

mapping = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar', 
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    10: 'Oct', 
    11: 'Nov',
    12: 'Dec'
}

sample['month'] = sample['month'].apply(lambda x: mapping[x])

fig = make_subplots()

gold_months = {'May', 'Jun', 'Jul'}

bar_colors = [
    'goldenrod' if m in gold_months else 'lightblue'
    for m in sample['month']
]

fig.add_trace(
    go.Bar(
        x=sample['month'],
        y=sample['Jordan'],
        name='Jordan Avg. Points',
        marker_color=bar_colors
    )
)

fig.add_trace(
    go.Scatter(x=sample['month'], y=sample['LeBron'], name='LeBron Avg. Points',
               mode='lines+markers'),
)

fig.add_trace(
    go.Bar(
        x=[None],
        y=[None],
        name='Playoff Months (May–July)',
        marker_color='goldenrod',
        showlegend=True
    )
)

fig.update_layout(title='Lebron Monthly Avg. Points vs. Jordan Avg. Points', height=500, width=800)

y_min = sample[['Jordan','LeBron']].min().min()
y_max = sample[['Jordan','LeBron']].max().max()

fig.update_yaxes(range=[y_min - 1, y_max + 1], secondary_y=False)
fig.update_yaxes(range=[y_min, y_max], secondary_y=True)

fig.update_xaxes(
    tickmode='array',
    tickvals=sample['month'],
    ticktext=sample['month']
)

fig.update_yaxes(title_text='Points')
fig.update_xaxes(title_text='Month')

fig.show()